<a href="https://colab.research.google.com/github/Vaibhavs10/scratchpad/blob/main/play_w_musicgen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 20.3 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
!git clone https://huggingface.co/music-gen-sprint/musicgen

Cloning into 'musicgen'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 76 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (76/76), 97.39 KiB | 461.00 KiB/s, done.


In [4]:
!cd musicgen && pip install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/musicgen
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 10.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 90.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 108.9 MB/s eta 0:00:00
     ━━━━━━━

In [5]:
from audiocraft.models import MusicGen

model = MusicGen.get_pretrained('small')

Downloading: "https://dl.fbaipublicfiles.com/audiocraft/musicgen/v0/b0dbef54-37d256b525.th" to /root/.cache/torch/hub/checkpoints/b0dbef54-37d256b525.th
100%|██████████| 225M/225M [00:11<00:00, 21.1MB/s]
Downloading: "https://dl.fbaipublicfiles.com/audiocraft/musicgen/v0/ba7a97ba-830fe5771e.th" to /root/.cache/torch/hub/checkpoints/ba7a97ba-830fe5771e.th
100%|██████████| 802M/802M [01:11<00:00, 11.8MB/s]


In [6]:
model.set_generation_params(
    use_sampling=True,
    top_k=250,
    duration=5
)

In [8]:
# Unconditional Generation
from audiocraft.utils.notebook import display_audio

output = model.generate_unconditional(num_samples=2, progress=False)
display_audio(output.detach().cpu().squeeze(), sample_rate=32000)

In [9]:
# Text Conditional Generation
from audiocraft.utils.notebook import display_audio

output = model.generate(
    descriptions=[
        '80s pop track with bassy drums and synth',
        '90s rock song with loud guitars and heavy drums',
    ],
    progress=False
)
display_audio(output[0].detach().cpu(), sample_rate=32000)
display_audio(output[1].detach().cpu(), sample_rate=32000)

In [10]:
# Melody conditioned generation
import torchaudio
from audiocraft.utils.notebook import display_audio

melody_waveform, sr = torchaudio.load("/home/felixkreuk/projs/wavs/bach_s1.mp3")
melody_waveform = melody_waveform.unsqueeze(0).repeat(2, 1, 1)
output = model.generate_with_chroma(
    descriptions=[
        '80s pop track with bassy drums and synth',
        '90s rock song with loud guitars and heavy drums',
    ],
    melody_wavs=melody_waveform,
    melody_sample_rate=sr,
    progress=False
)
display_audio(output[0].detach().cpu(), sample_rate=32000)
display_audio(output[1].detach().cpu(), sample_rate=32000)

RuntimeError: ignored